In [1]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["PYTHONHASHSEED"] = "0"
#os.environ['TF_DETERMINISTIC_OPS'] = '1' 
import tensorflow as tf
import random

In [2]:
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:
print(tf.__version__)

2.9.0


In [4]:
dataset_name='gowalla'
dataset_folder='gowalla'
preflix_folder='24_07_30'

In [5]:
import pickle
import platform


items_num=5855 
users_num=8710 


batch_size=512
epoch_num=1000
hidden_size=64
keep_rate=0.9
layers_num=2
neg_layers_num=1
loss_ratio=0.01

neg_num=1
test_neg_num=1000

In [6]:
print(items_num)

5855


In [7]:
if platform.system().lower() == 'windows':
    print("windows")
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_train_data_index_from_0.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_test_data_index_from_0.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_val_data_index_from_0.pkl', 'rb') as f:
        val_data=pickle.load(f)
elif platform.system().lower() == 'linux':
    print("linux")
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_train_data_index_from_0.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_test_data_index_from_0.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_val_data_index_from_0.pkl', 'rb') as f:
        val_data=pickle.load(f)

linux


In [8]:
print(np.shape(train_data['pos_item']),np.shape(test_data['pos_item']),np.shape(val_data['pos_item']))

(35994,) (9496,) (4729,)


In [9]:
users_num_max=max(max(train_data['users']),max(test_data['users']),max(val_data['users']))
users_num_min=min(min(train_data['users']),min(test_data['users']),min(val_data['users']))
items_num_max=max(max(train_data['pos_item']),max(test_data['pos_item']),max(val_data['pos_item']))
items_num_min=min(min(train_data['pos_item']),min(test_data['pos_item']),min(val_data['pos_item']))
print('users range:[',users_num_min,',',users_num_max,']')
print('items range:[',items_num_min,',',items_num_max,']')

users range:[ 0 , 8709 ]
items range:[ 0 , 5854 ]


In [10]:
import scipy.sparse as sp
import copy
if platform.system().lower() == 'windows':
    print("windows")
    A_norm = sp.load_npz('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_pre_adj_mat_index_from_0.npz')#D^-1/2*A*D^-1/2
    L_norm = sp.load_npz('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_l_norm_3_prob_mat.npz')#I-D^-1/2*A*D^-1/2
elif platform.system().lower() == 'linux':
    print("linux")
    A_norm = sp.load_npz('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_pre_adj_mat_index_from_0.npz')#D^-1/2*A*D^-1/2
    L_norm = sp.load_npz('/root/autodl-nas/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_l_norm_3_prob_mat.npz')

L_norm=L_norm-sp.csr_matrix(np.identity(items_num+users_num))
print('already load adj matrix', A_norm.shape)
print('already load adj matrix', L_norm.shape)
print(A_norm.dtype)
print(L_norm.dtype)
def _preprocess_sparse_matrix(X):
    coo = X.tocoo().astype(np.float32)
    indices = np.vstack((coo.row, coo.col)).astype(np.int64).T
    values = coo.data
    dense_shape = coo.shape
    return indices, values, dense_shape
A_norm=_preprocess_sparse_matrix(A_norm)
L_norm=_preprocess_sparse_matrix(L_norm)
ori_A_norm=copy.copy(A_norm)
ori_L_norm=copy.copy(L_norm)

linux
already load adj matrix (14565, 14565)
already load adj matrix (14565, 14565)
float32
float64


In [13]:
class lightgcn_layer(tf.keras.layers.Layer):
    def pos_convolution(self,adj,x,layers_num,training):
        all_embeddings=[x]
        for k in range(0, layers_num):
            x = tf.sparse.sparse_dense_matmul(adj, x)
            if training:
                x=self.dropout(x)      
            all_embeddings += [x]  
        all_embeddings = tf.stack(all_embeddings, 1)
        all_embeddings = tf.reduce_mean(input_tensor=all_embeddings, axis=1, keepdims=False)
        return all_embeddings

    def neg_deconvolution(self,adj,x,layers_num,training):
        all_embeddings=[x]
        for k in range(0, layers_num):
            x =tf.sparse.sparse_dense_matmul(adj, x)
            if training:
                x=self.dropout(x)      
            all_embeddings += [x]  
        all_embeddings = tf.stack(all_embeddings, 1)
        all_embeddings = tf.reduce_mean(input_tensor=all_embeddings, axis=1, keepdims=False)
        return all_embeddings



    def __init__(self,hidden_size=64, dropout=0., embed_reg=1e-6):
        super(lightgcn_layer, self).__init__()




        initializer_p = tf.keras.initializers.RandomNormal(mean=0., stddev=0.05,seed=2024)
        initializer_n = tf.keras.initializers.RandomNormal(mean=0., stddev=0.05,seed=2023)

        self.item_embedding_pool = tf.keras.layers.Embedding(items_num,hidden_size,input_length=items_num,
                                                       embeddings_initializer=initializer_p,
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.user_embedding_pool = tf.keras.layers.Embedding(users_num,hidden_size,input_length=users_num,
                                                       embeddings_initializer=initializer_p,
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))

        self.neg_item_embedding_pool = tf.keras.layers.Embedding(items_num,hidden_size,input_length=items_num,
                                                       embeddings_initializer=initializer_n,
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.neg_user_embedding_pool = tf.keras.layers.Embedding(users_num,hidden_size,input_length=users_num,
                                                       embeddings_initializer=initializer_n,
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))

        self.dropout = tf.keras.layers.Dropout(dropout)


    def call(self, inputs,A_norm,L_norm,build_graph=False,training=False):
        A_hat=A_norm
        L_hat=L_norm

        self.item_embedding=self.item_embedding_pool(tf.range(items_num))
        self.user_embedding=self.user_embedding_pool(tf.range(users_num))

        self.neg_item_embedding=self.neg_item_embedding_pool(tf.range(items_num))
        self.neg_user_embedding=self.neg_user_embedding_pool(tf.range(users_num))


        all_embeddings = tf.concat([self.user_embedding,self.item_embedding], axis=0)
        neg_all_embeddings=tf.concat([self.neg_user_embedding,self.neg_item_embedding], axis=0)


        g1_all_embeddings = self.pos_convolution(A_hat,all_embeddings,layers_num,training)
        g2_all_embeddings = self.neg_deconvolution(L_hat,neg_all_embeddings,neg_layers_num,training)


        self.g1_ua_embeddings, self.g1_ia_embeddings = tf.split(
            g1_all_embeddings, [users_num,items_num], 0
        )

        self.g2_ua_embeddings, self.g2_ia_embeddings = tf.split(
            g2_all_embeddings, [users_num,items_num], 0
        )     

        return tf.reshape(self.g1_ua_embeddings,[-1,hidden_size]),tf.reshape(self.g1_ia_embeddings,[-1,hidden_size]),\
    tf.reshape(self.g2_ua_embeddings,[-1,hidden_size]),tf.reshape(self.g2_ia_embeddings,[-1,hidden_size])

In [15]:
class lightgcn(tf.keras.models.Model):

    def _convert_sp_mat_to_sp_tensor(self, mat):
        """Convert preprocessed data to tf.SparseTensor.
        Returns:
            tf.SparseTensor: SparseTensor after conversion.
        """
        indices, values, dense_shape=mat
        indices = tf.convert_to_tensor(indices, dtype=tf.int64)
        values = tf.convert_to_tensor(values, dtype=tf.float32)
        dense_shape = tf.convert_to_tensor(dense_shape, dtype=tf.int64)
        
        sp_tensor = tf.SparseTensor(indices, values, dense_shape)
        sp_tensor = tf.sparse.reorder(sp_tensor)
        
        return sp_tensor

    def cal_loss(self,u,p,n,test_mask,full=False):
        pos_score = tf.reduce_sum(tf.multiply(u, p), axis=-1) # (None, 1)
        neg_score = tf.reduce_sum(tf.multiply(u, n), axis=-1)
        
        
        if full:
            neg_score=tf.where(neg_score == 0,-1e10,neg_score)
        
        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
       
        loss = tf.reduce_mean(- tf.math.log(tf.math.sigmoid(pos_score-neg_score)))
        return loss,logits
    
    

    def cosine_similarity(self,a, b):
        normalize_a = tf.math.l2_normalize(a,1)        
        normalize_b = tf.math.l2_normalize(b,1)
        cos_similarity=tf.linalg.matmul(normalize_a,normalize_b,transpose_b=True)
        return cos_similarity
    
    
    
    
    def cal_cl_loss(self,view1,view2,temp):
        cs=self.cosine_similarity(view1,view2)
        pos_cs=tf.linalg.diag_part(cs)
        loss=tf.math.reduce_mean(- tf.math.log(tf.math.exp(pos_cs/temp)/tf.math.reduce_sum(tf.math.exp(cs/temp),axis=1)))
        return loss
    

    def __init__(self):
        super(lightgcn, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        self.print_tensor_g1_u = tf.keras.layers.Lambda(print_tensor_g1_u)
        self.print_tensor_g1_i = tf.keras.layers.Lambda(print_tensor_g1_i)
        self.print_tensor_g2_u = tf.keras.layers.Lambda(print_tensor_g2_u)
        self.print_tensor_g2_i = tf.keras.layers.Lambda(print_tensor_g2_i)
        self.u_project=tf.keras.layers.Dense(hidden_size)
        self.i_project=tf.keras.layers.Dense(hidden_size)



        self.lg_layer=lightgcn_layer(64,0.25,1e-6)
        self.use_l2norm=use_l2norm
        self.temp=0.5
        
    def call(self,inputs,A_hat,L_hat,training=False,full=False):
        
        if isinstance(A_hat, tf.sparse.SparseTensor) or isinstance(L_hat, tf.sparse.SparseTensor):
            A_hat=A_hat
            L_hat=L_hat
            
        else:
            A_hat=self._convert_sp_mat_to_sp_tensor(A_hat)
            L_hat=self._convert_sp_mat_to_sp_tensor(L_hat) 
        
        self.g1_ua_embeddings, self.g1_ia_embeddings,\
        self.g2_ua_embeddings, self.g2_ia_embeddings=self.lg_layer(inputs,A_hat,L_hat,training)
   
        g1_u = tf.gather(
            params=self.g1_ua_embeddings, indices=tf.reshape(inputs['users'],[1,-1])
        )
        g1_p = tf.gather(
            params=self.g1_ia_embeddings, indices=tf.reshape(inputs['pos_item'],[1,-1])
        )
        
        g2_u = tf.gather(
            params=self.g2_ua_embeddings, indices=tf.reshape(inputs['users'],[1,-1])
        )
        g2_p = tf.gather(
            params=self.g2_ia_embeddings, indices=tf.reshape(inputs['pos_item'],[1,-1])
        )
        
        indices = tf.tile(tf.reshape(tf.range(items_num), [1, items_num]),[tf.shape(inputs['pos_item'])[0],1])
        pos_idx_expanded = tf.tile(tf.reshape(inputs['pos_item'],[-1,1]), (1, items_num))
        test_mask = tf.math.not_equal(indices, pos_idx_expanded)#[b,1430]
        
        
        if full==True:
            g1_n=tf.expand_dims(self.g1_ia_embeddings,1)
            g2_n=tf.expand_dims(self.g2_ia_embeddings,1)
        else:
            neg_indx=tf.random.uniform([tf.shape(inputs['pos_item'])[0],neg_num],dtype=tf.int32,maxval=items_num,minval=0,seed=2023)
            pos_idx = tf.repeat(tf.reshape(inputs['users'],[-1,1]), neg_num, axis=1)
            mask = tf.equal(neg_indx, pos_idx)
            neg_indx = tf.where(mask, tf.random.uniform([tf.shape(inputs['pos_item'])[0],neg_num],dtype=tf.int32,maxval=items_num,minval=0,seed=2024), neg_indx)   
            g1_n = tf.gather(
                params=self.g1_ia_embeddings, indices=tf.reshape(neg_indx,[tf.shape(neg_indx)[1],-1])
            )
            g2_n = tf.gather(
                params=self.g2_ia_embeddings, indices=tf.reshape(neg_indx,[tf.shape(neg_indx)[1],-1])
            )
        g1_u=tf.transpose(g1_u,[1,0,2])#[b,1,h]
        g1_p=tf.transpose(g1_p,[1,0,2])
        g1_n=tf.transpose(g1_n,[1,0,2])
        
        g2_u=tf.transpose(g2_u,[1,0,2])
        g2_p=tf.transpose(g2_p,[1,0,2])
        g2_n=tf.transpose(g2_n,[1,0,2])#[b,neg_num,h]

        
        if self.use_l2norm:
            g1_u = tf.math.l2_normalize(g1_u, axis=-1)
            g1_p = tf.math.l2_normalize(g1_p, axis=-1)
            g1_n = tf.math.l2_normalize(g1_n, axis=-1)
            g2_u = tf.math.l2_normalize(g2_u, axis=-1)
            g2_p = tf.math.l2_normalize(g2_p, axis=-1)
            g2_n = tf.math.l2_normalize(g2_n, axis=-1)
        

        l1,logits1=self.cal_loss(g1_u,g1_p,g1_n,test_mask,full)
        l2,logits2=self.cal_loss(g1_u,g1_p,g2_n,test_mask,full)
        l3,logits3=self.cal_loss(g1_u,g2_p,g1_n,test_mask,full)
        l4,logits4=self.cal_loss(g1_u,g2_p,g2_n,test_mask,full)
        
        l5,logits5=self.cal_loss(g2_u,g1_p,g1_n,test_mask,full)
        l6,logits6=self.cal_loss(g2_u,g1_p,g2_n,test_mask,full)
        l7,logits7=self.cal_loss(g2_u,g2_p,g1_n,test_mask,full)
        l8,logits8=self.cal_loss(g2_u,g2_p,g2_n,test_mask,full)
        
        bpr_loss=l1+l4+l5+l8
        cl_loss=self.cal_cl_loss(self.g1_ua_embeddings,self.g2_ua_embeddings,self.temp)+\
        self.cal_cl_loss(self.g1_ia_embeddings,self.g2_ia_embeddings,self.temp)
        
        
        logits=tf.nn.softmax(logits1+logits4+logits5+logits8,axis=-1)
        
        return logits,(bpr_loss,cl_loss),(self.g1_ua_embeddings, self.g1_ia_embeddings,self.g2_ua_embeddings, self.g2_ia_embeddings)
    
    
    def summary(self):
        inputs = {
            'users': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32)
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [16]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        elif name == 'precision':
            res = hr(rank, 1)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data,A_norm,L_norm, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    print('start testing!')
    test_dataset = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)
    final_dict={}
    for i in metric_names:
        final_dict.update({i:[]})
    for step, batch_test in enumerate(test_dataset):
        pred_y,_,_ =  model(batch_test,A_norm,L_norm,training=False,full=True)
        pred_y=-pred_y.numpy()
        #print(np.shape(pred_y))
        part_dict=eval_rank(pred_y, metric_names, k)
        for i in metric_names:
            final_dict [i].append(part_dict[i])
    for i in metric_names:
        final_dict [i]=np.mean(final_dict [i])
    return final_dict

In [17]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

In [18]:
@tf.function
def dynamic_update(A_hat, L_hat,g1_ua_embeddings,g1_ia_embeddings,g2_ua_embeddings,g2_ia_embeddings):
    def _convert_sp_mat_to_sp_tensor(mat):
        """Convert preprocessed data to tf.SparseTensor.
        Returns:
            tf.SparseTensor: SparseTensor after conversion.
        """
        indices, values, dense_shape=mat
        indices = tf.convert_to_tensor(indices, dtype=tf.int64)
        values = tf.convert_to_tensor(values, dtype=tf.float32)
        dense_shape = tf.convert_to_tensor(dense_shape, dtype=tf.int64)
        
        sp_tensor = tf.SparseTensor(indices, values, dense_shape)
        sp_tensor = tf.sparse.reorder(sp_tensor)
        
        return sp_tensor
    def tfdense_to_sparse(dense_matrix):
        indices = tf.where(tf.not_equal(dense_matrix, 0))
        values = tf.gather_nd(dense_matrix, indices)
        sparse_matrix = tf.SparseTensor(indices, values, dense_matrix.shape)
        return sparse_matrix


    def build_graph_function(adj,topk):
        knn_val, knn_ind = tf.math.top_k(adj, k=topk, sorted=True)
        weighted_adjacency_matrix = tf.zeros_like(adj)
        batch_indices = tf.range(tf.shape(adj)[0])[:, None]
        batch_indices = tf.tile(batch_indices, [1, topk])
        scatter_indices = tf.stack([batch_indices, knn_ind], axis=-1)
        weighted_adjacency_matrix = tf.tensor_scatter_nd_update(weighted_adjacency_matrix, scatter_indices, knn_val)
        
        b_non_zero = tf.where(adj == 0, tf.fill(adj.shape, float('inf')), adj)
        indices = tf.argsort(b_non_zero, axis=1)[:, :topk]
        mask = tf.ones_like(adj)
        row_indices = tf.range(adj.shape[0])[:, tf.newaxis]
        full_indices = tf.concat([tf.tile(row_indices, [1, topk]), indices], axis=-1)
        full_indices = tf.reshape(full_indices, [-1, 2])
        mask = tf.tensor_scatter_nd_update(mask, full_indices, tf.zeros([adj.shape[0] * topk]))
        
        return weighted_adjacency_matrix,mask



    def compute_normalized_laplacian(adj):
        rowsum = tf.reduce_sum(adj, axis=-1)
        d_inv_sqrt = tf.math.pow(rowsum, -0.5)
        d_inv_sqrt = tf.where(tf.math.is_inf(d_inv_sqrt), tf.zeros_like(d_inv_sqrt), d_inv_sqrt)
        d_mat_inv_sqrt = tf.linalg.diag(d_inv_sqrt)
        L_norm = tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

        return L_norm



    def r2a(rating):
        m, n = rating.shape
        zero_matrix_m = tf.zeros((m, m), dtype=rating.dtype)
        aa = tf.concat([zero_matrix_m, rating], axis=1)
        zero_matrix_n = tf.zeros((n, n), dtype=rating.dtype)
        a_transpose = tf.transpose(rating)
        aaa = tf.concat([a_transpose, zero_matrix_n], axis=1)
        combined_matrix = tf.concat([aa, aaa], axis=0) 
        return combined_matrix


    def cosine_similarity(a, b):
        normalize_a = tf.math.l2_normalize(a,1)        
        normalize_b = tf.math.l2_normalize(b,1)
        cos_similarity=tf.linalg.matmul(normalize_a,normalize_b,transpose_b=True)
        return cos_similarity
    tf.print('update graph')
    if isinstance(A_hat, tf.sparse.SparseTensor) or isinstance(L_hat, tf.sparse.SparseTensor):
        A_hat=A_hat
        L_hat=L_hat
    else:
        A_hat=_convert_sp_mat_to_sp_tensor(A_hat)
        L_hat=_convert_sp_mat_to_sp_tensor(L_hat)
    g1_learned_n_r,g1_n_mask = build_graph_function(-cosine_similarity(g1_ua_embeddings, g1_ia_embeddings), 5)
    g1_learned_n_adj = r2a(g1_learned_n_r)
    g1_n_mask=r2a(g1_n_mask)
    g1_learned_n_norm_adj = tfdense_to_sparse(compute_normalized_laplacian(g1_learned_n_adj))

    g2_learned_n_r,g2_n_mask = build_graph_function(-cosine_similarity(g2_ua_embeddings, g2_ia_embeddings), 5)
    g2_learned_n_adj = r2a(g2_learned_n_r)
    g2_n_mask=r2a(g2_n_mask)
    g2_learned_n_norm_adj = tfdense_to_sparse(compute_normalized_laplacian(g2_learned_n_adj))
    
    n_mask = tf.where((g1_n_mask == 0.) & (g2_n_mask == 0.), 0., 1.)
    learned_n_norm_adj = tf.sparse.add(g1_learned_n_norm_adj, g2_learned_n_norm_adj)
    L_hat=L_hat*n_mask
    L_hat = tf.sparse.add(tf.sparse.map_values(lambda x: x * 0.7, L_hat), tf.sparse.map_values(lambda x: x * 0.3, learned_n_norm_adj))

    g1_learned_p_r,g1_p_mask = build_graph_function(cosine_similarity(g1_ua_embeddings, g1_ia_embeddings), 5)
    g1_learned_p_adj = r2a(g1_learned_p_r)
    g1_p_mask=r2a(g1_p_mask)
    g1_learned_p_norm_adj = tfdense_to_sparse(compute_normalized_laplacian(g1_learned_p_adj))

    g2_learned_p_r,g2_p_mask = build_graph_function(cosine_similarity(g2_ua_embeddings, g2_ia_embeddings), 5)
    g2_learned_p_adj = r2a(g2_learned_p_r)
    g2_p_mask=r2a(g2_p_mask)
    g2_learned_p_norm_adj = tfdense_to_sparse(compute_normalized_laplacian(g2_learned_p_adj))
    p_mask = tf.where((g1_p_mask == 0.) & (g2_p_mask == 0.), 0., 1.)
    learned_p_norm_adj = tf.sparse.add(g1_learned_p_norm_adj, g2_learned_p_norm_adj)
    A_hat=A_hat*p_mask
    A_hat = tf.sparse.add(tf.sparse.map_values(lambda x: x * 0.8, A_hat), tf.sparse.map_values(lambda x: x * 0.2, learned_p_norm_adj))
    return A_hat, L_hat

In [19]:
from time import time
from tensorflow.keras.backend import clear_session
import tensorflow as tf

number = 1
max_layers = 10
ave_dict = {'hr': [], 'mrr': [], 'ndcg': [], 'precision': []}
tb = time()

train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data).batch(batch_size)

update_interval = int(len(train_dataset) / 1)
epoch_interval=2

@tf.function
def train_step(batch_train, model,optimizer,A_norm,L_norm,loss_ratio, update_inte, step): 
    build_graph = step % update_inte == 0
    with tf.GradientTape() as tape:
        logits, (bpr_loss, cl_loss),(g1_ua_embeddings,g1_ia_embeddings,g2_ua_embeddings,g2_ia_embeddings)\
        = model(batch_train,A_norm,L_norm, training=True, full=False)
        loss_value = bpr_loss + loss_ratio * cl_loss
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value, bpr_loss, cl_loss, g1_ua_embeddings,g1_ia_embeddings,g2_ua_embeddings,g2_ia_embeddings

@tf.function
def val_step(model,batch_val,A_norm,L_norm):
    val_logits, val_loss,_ = model(batch_val,A_norm,L_norm, training=False, full=False)
    return val_loss[0]



model = lightgcn()
#model.summary()
for callback in callbacks:
    callback.set_model(model)
    callback.on_train_begin(logs={})

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=optimizer)

for epoch in range(epoch_num):
    total_train_loss = 0.
    total_bpr_loss = 0.
    total_cl_loss = 0.
    num_train_batches = 0.
    total_val_loss = 0.
    num_val_batches = 0.
    print(f"Epoch {epoch + 1}/{epoch_num}")
    t1 = time() 
    for callback in callbacks:
        callback.on_epoch_begin(epoch, logs={})

    for step, batch_train in enumerate(train_dataset):
        loss_value, bpr_loss, cl_loss,\
        g1_ua_embeddings,g1_ia_embeddings,g2_ua_embeddings,g2_ia_embeddings= train_step(batch_train, model,optimizer,A_norm,L_norm,tf.constant(loss_ratio), tf.constant(update_interval), tf.constant(step))
        #loss_value, bpr_loss, cl_loss = train_step(batch_train, model,optimizer,loss_ratio, update_inte, step)
        if (epoch%epoch_interval==0) and ((step+1)%update_interval==0):
            A_norm,L_norm=dynamic_update(ori_A_norm,ori_L_norm,g1_ua_embeddings,g1_ia_embeddings,g2_ua_embeddings,g2_ia_embeddings)        
        total_train_loss += loss_value
        total_bpr_loss += bpr_loss
        total_cl_loss += loss_ratio * cl_loss
        num_train_batches += 1

    avg_train_loss = total_train_loss / num_train_batches
    avg_bpr_loss = total_bpr_loss / num_train_batches
    avg_cl_loss = total_cl_loss / num_train_batches
    print(f"Time cost: {time()-t1:.2f}, Training loss: {avg_train_loss.numpy():.4f}, bpr loss: {avg_bpr_loss.numpy():.4f}, cl loss: {avg_cl_loss.numpy():.4f}")
    t2=time()
    for batch_val in val_dataset:
        val_loss = val_step(model,batch_val,A_norm,L_norm)
        total_val_loss += val_loss
        num_val_batches += 1

    avg_val_loss = total_val_loss / num_val_batches
    print(f"Time cost: {time()-t2:.2f}, Validation loss: {avg_val_loss.numpy():.4f}")

    for callback in callbacks:
        callback.on_epoch_end(epoch, logs={'val_loss': avg_val_loss})

    if any([callback.stopped_epoch for callback in callbacks]):
        break

for callback in callbacks:
    callback.on_train_end()
eval_dict = eval_pos_neg(model, test_data,A_norm,L_norm, ['hr', 'mrr', 'ndcg', 'precision'], 10, batch_size * 2)
for m in ['hr', 'mrr', 'ndcg', 'precision']:
    ave_dict[m].append(eval_dict[m])
print('Precision = %.4f, HR = %.4f, MRR = %.4f, NDCG = %.4f'
      % (eval_dict['precision'], eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
te = time()
print('Fit [%.1f s]: Precision = %.4f, HR = %.4f, MRR = %.4f, NDCG = %.4f'
      % (te - tb, np.mean(ave_dict['precision']), np.mean(ave_dict['hr']), np.mean(ave_dict['mrr']), np.mean(ave_dict['ndcg'])))

Epoch 1/1000
update graph
Time cost: 13.09, Training loss: 2.9316, bpr loss: 2.7636, cl loss: 0.1680
Time cost: 0.55, Validation loss: 2.7706
Epoch 2/1000
Time cost: 2.94, Training loss: 2.8931, bpr loss: 2.7361, cl loss: 0.1570
Time cost: 0.03, Validation loss: 2.7555
Epoch 3/1000
update graph
Time cost: 2.69, Training loss: 2.8290, bpr loss: 2.6770, cl loss: 0.1520
Time cost: 0.04, Validation loss: 2.6782
Epoch 4/1000
Time cost: 1.03, Training loss: 2.6481, bpr loss: 2.4954, cl loss: 0.1527
Time cost: 0.03, Validation loss: 2.5064
Epoch 5/1000
update graph
Time cost: 2.64, Training loss: 2.4072, bpr loss: 2.2537, cl loss: 0.1535
Time cost: 0.04, Validation loss: 2.3293
Epoch 6/1000
Time cost: 1.07, Training loss: 2.1851, bpr loss: 2.0316, cl loss: 0.1535
Time cost: 0.04, Validation loss: 2.1664
Epoch 7/1000
update graph
Time cost: 2.71, Training loss: 1.9925, bpr loss: 1.8388, cl loss: 0.1536
Time cost: 0.05, Validation loss: 2.0520
Epoch 8/1000
Time cost: 1.01, Training loss: 1.8418